In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'Comp4651/Project/finadvisor-serverless'
PROJECT_PATH = f'/content/drive/MyDrive/{FOLDERNAME}'

import os, sys
os.chdir(PROJECT_PATH)

sys.path.append(PROJECT_PATH)

Mounted at /content/drive


In [11]:
import json
import random
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer
from tabulate import tabulate  # pip install tabulate

# Resolve local model path
model_path = Path("aws/package/model").resolve()

# Load tokenizer (same for both models)
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small", legacy=True)

# Load trained model from local directory
trained_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Load untrained baseline model from Hugging Face Hub
baseline_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

# Read evaluation prompts
with open("data_gen/training/eval_prompts.jsonl") as f:
    lines = f.readlines()

# Sample N random prompts
sample_size = 10
random.seed(42)
sampled_lines = random.sample(lines, sample_size)

# Collect results
results = []
for line in sampled_lines:
    prompt = json.loads(line)["input"]

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate with trained model
    trained_outputs = trained_model.generate(**inputs, max_length=128)
    trained_suggestion = tokenizer.decode(trained_outputs[0], skip_special_tokens=True)

    # Generate with baseline model
    baseline_outputs = baseline_model.generate(**inputs, max_length=128)
    baseline_suggestion = tokenizer.decode(baseline_outputs[0], skip_special_tokens=True)

    # Append row
    results.append([prompt, trained_suggestion, baseline_suggestion])

# Print results in table format
headers = ["Prompt", "Trained Suggestion", "Baseline Suggestion"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Prompt                                                                                  | Trained Suggestion                                                                                                                                                                                                                      | Baseline Suggestion                                                                                                                                           |
+===========================